In [1]:
import logging
reload(logging)
log_fmt = '%(asctime)-9s %(levelname)-8s: %(message)s'
logging.basicConfig(format=log_fmt)

# Change to info once the notebook runs ok
#logging.getLogger().setLevel(logging.DEBUG)

In [2]:
%pylab inline

import datetime
import devlib
import os
import json
import pandas as pd
import re
import subprocess
import trappy
from trappy.plotter.Utils import get_trace_event_data

import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt

# Support to access the remote target
#import devlib
#from env import TestEnv

from executor import Executor

Populating the interactive namespace from numpy and matplotlib


# Target Configuration

In [3]:
# Setup a target configuration
my_target_conf = {
    
    # Target platform and board
    "platform"    : 'linux',
    "board"       : 'aboard',
    
    # Target board IP/MAC address
    "host"        : '192.168.0.1',
    
    # Login credentials
    "username"    : 'root',
    "password"    : 'test0000',

}

# Tests Configuration

In [4]:
my_tests_conf = {

    # Folder where all the results will be collected
    "results_dir" : "ExecutorExample",

    # Platform configurations to test
    "confs" : [
        {
            "tag"            : "base",
            "flags"          : "ftrace",           # Enable FTrace events
            "sched_features" : "NO_ENERGY_AWARE",  # Disable EAS
            "cpufreq"        : {                   # Use PERFORMANCE CpuFreq
                "governor" : "performance",
            },
        },
        {
            "tag"            : "eas",
            "flags"          : "ftrace",           # Enable FTrace events
            "sched_features" : "ENERGY_AWARE",     # Enable EAS
            "cpufreq"        : {                   # Use PERFORMANCE CpuFreq
                "governor" : "performance",
            },
        },
    ],
    
    # Workloads to run (on each platform configuration)
    "wloads" : {
        # Run hackbench with 1 group using pipes
        "perf" : {
            "type" : "perf_bench",
            "conf" : {
                "class" : "messaging",
                "params" : {
                    "group" :    1,
                    "loop"  :   10,
                    "pipe"  : True,
                    "thread": True,
                }
            }
        },
        # Run a 20% duty-cycle periodic task
        "rta" : {
            "type" : "rt-app",
            "loadref" : "big",
            "conf" : {
                "class"  : "profile",
                "params"  : {
                    "p20" : {
                        "kind"   : "periodic",
                        "params" : {
                            "duty_cycle_pct" : 20,
                         },
                    },
                },
            },
        },
    },
    
    # Number of iterations for each workload
    "iterations" : 1,
    
    # FTrace events to collect for all the tests configuration which have
    # the "ftrace" flag enabled
    "ftrace"  : {
         "events" : [
            "sched_switch",
            "sched_wakeup",
            "sched_wakeup_new",
            "cpu_frequency",
         ],
         "buffsize" : 80 * 1024,
    },
    
    # Tools required by the experiments
    "tools"   : [ 'trace-cmd', 'perf' ],
    
    # Modules required by these experiments
    "modules"     : [ 'bl', 'cpufreq' ],

}

# Tests execution

In [5]:
executor = Executor(my_target_conf, my_tests_conf)

10:22:10  INFO    :         Target - Loading custom (inline) test configuration
10:22:10  INFO    :         Target - Using base path: /home/derkling/Code/lisa
10:22:10  INFO    :         Target - Loading custom (inline) target configuration
10:22:10  INFO    :         Target - Loading custom (inline) test configuration
10:22:10  INFO    :         Target - Devlib modules to load: ['bl', 'cpufreq']
10:22:10  INFO    :         Target - Connecting linux target:
10:22:10  INFO    :         Target -   username : root
10:22:10  INFO    :         Target -       host : 192.168.0.1
10:22:10  INFO    :         Target -   password : test0000
10:22:15  INFO    :         Target - Initializing target workdir:
10:22:15  INFO    :         Target -    /root/devlib-target
10:22:21  INFO    :         Target - Topology:
10:22:21  INFO    :         Target -    [[0, 3, 4, 5], [1, 2]]
10:22:25  INFO    :         FTrace - Enabled tracepoints:
10:22:25  INFO    :         FTrace -   sched_switch
10:22:25  INFO  

In [6]:
executor.run()

10:22:25  INFO    : 
10:22:25  INFO    : ################################################################################
10:22:25  INFO    :       Executor - Experiments execution
10:22:25  INFO    : ################################################################################
10:22:25  INFO    : 
10:22:25  INFO    : ================================================================================
10:22:25  INFO    :   TargetConfig - configuring target for [base] experiments
10:22:27  INFO    :  SchedFeatures - Set scheduler feature: NO_ENERGY_AWARE
10:22:27  INFO    :        CPUFreq - Configuring all CPUs to use [performance] governor
10:22:27  INFO    :          WlGen - Setup new workload rta
10:22:27  INFO    :          RTApp - Workload duration defined by longest task
10:22:27  INFO    :          RTApp - Default policy: SCHED_OTHER
10:22:27  INFO    :          RTApp - ------------------------
10:22:27  INFO    :          RTApp - task [task_p20], sched: using default policy
10:22

In [7]:
!tree {executor.te.res_dir}

/home/derkling/Code/lisa/results/ExecutorExample
├── perf_bench_messaging:base:perf
│   ├── 1
│   │   ├── output.log
│   │   ├── performance.json
│   │   └── trace.dat
│   ├── kernel.config
│   ├── kernel.version
│   └── platform.json
├── perf_bench_messaging:eas:perf
│   ├── 1
│   │   ├── output.log
│   │   ├── performance.json
│   │   └── trace.dat
│   ├── kernel.config
│   ├── kernel.version
│   └── platform.json
├── rtapp:base:rta
│   ├── 1
│   │   ├── output.log
│   │   ├── rta_00.json
│   │   ├── rt-app-task_p20-0.log
│   │   └── trace.dat
│   ├── kernel.config
│   ├── kernel.version
│   └── platform.json
└── rtapp:eas:rta
    ├── 1
    │   ├── output.log
    │   ├── rta_00.json
    │   ├── rt-app-task_p20-0.log
    │   └── trace.dat
    ├── kernel.config
    ├── kernel.version
    └── platform.json

8 directories, 26 files
